In [1]:
import numpy as np 
import pandas as pd
import datetime
import calendar
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
X_train = pd.read_csv('../data/X_train.csv', sep=',', encoding='utf-8')

In [3]:
print(len(X_train))
X_train['sepostulo'].head(2)

13097808


0    False
1    False
Name: sepostulo, dtype: bool

In [4]:
# Desordenamos para que no queden todos los true primero y los false despues
X_train = X_train.sample(len(X_train))

In [5]:
X_train['sepostulo'].head(10)

1384709      True
8618296     False
4650534      True
9817466      True
1759294      True
3433121     False
2866353      True
11834995     True
4632593      True
6504418      True
Name: sepostulo, dtype: bool

In [6]:
target = X_train[['sepostulo']]

In [7]:
X_train.to_csv('../data/X_train.csv', index=False, sep=',', encoding='utf-8')
target.to_csv('../data/target.csv', index=False, sep=',', encoding='utf-8')

In [15]:
target = pd.read_csv('../data/target.csv', sep=',', encoding='utf-8')

In [2]:
X_train = pd.read_csv('../data/X_train.csv', sep=',', encoding='utf-8')

In [19]:
X_test = pd.read_csv('../data/X_test.csv', sep=',', encoding='utf-8')

In [20]:
X_test.shape

(100120, 55)

In [4]:
print('train shape:', X_train.shape,'| target shape:', target.shape)
print('test shape:', X_test.shape)

train shape: (13097808, 56) | target shape: (13097808, 1)
test shape: (100120, 55)


In [21]:
X_test.duplicated('id').any()

True

In [23]:
X_test.head(1)

,id,idaviso,idpostulante,edad,nombre_estudio_Doctorado,nombre_estudio_Master,nombre_estudio_Otro,nombre_estudio_Posgrado,nombre_estudio_Secundario,nombre_estudio_Terciario/Tecnico,nombre_estudio_Universitario,estado_estudio_Abandonado,estado_estudio_EnCurso,estado_estudio_Graduado,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,nombre_area,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,cant_vistas_aviso,cant_postulaciones_aviso,tipo_full,tipo_part,tipo_otro,nivel_laboral_Gerencia/AltaGerencia/Dirección,nivel_laboral_Jefe/Supervisor/Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior/Semi-Senior,Ventas,Administración,Producción,Comercial,AtenciónalCliente,Recepcionista,CallCenter,Tesorería,Telemarketing,MantenimientoyLimpieza,Almacén/Depósito/Expedición,Contabilidad,RecursosHumanos,Gastronomia,Logística,Secretaria,AdministracióndePersonal,CréditosyCobranzas,Marketing,Legal
0,0,739260,6M9ZQR,42,1,0,0,0,0,0,0,0,0,1,1,0,0,Comercial,0,0,0,1,0,0,0,2,0,True,False,False,0,1,0,0,0,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [25]:
X_test.drop_duplicates(['idaviso', 'idpostulante'], inplace=True, keep='last')
X_test.shape

(100000, 55)

In [26]:
ids = X_test[['id']]

In [27]:
X_test.drop('id', axis=1, inplace=True)

In [7]:
print('train shape:', X_train.shape,'| target shape:', target.shape)
print('test shape:', X_test.shape)

train shape: (13097808, 56) | target shape: (13097808, 1)
test shape: (100120, 54)


In [28]:
ids.to_csv('test_.csv', sep=',', encoding='utf-8')

In [6]:
print(X_train.shape)
print(X_test.shape)

(13097808, 56)
(100120, 54)


In [10]:
print(len(target[target['sepostulo'] == False]))
print(len(target[target['sepostulo'] == True]))

6495985
6601823


In [7]:
X_train.head(1)

,Administración,AdministracióndePersonal,Almacén/Depósito/Expedición,AtenciónalCliente,CallCenter,Comercial,Contabilidad,CréditosyCobranzas,Gastronomia,Legal,Logística,MantenimientoyLimpieza,Marketing,Producción,Recepcionista,RecursosHumanos,Secretaria,Telemarketing,Tesorería,Ventas,cant_postulaciones_aviso,cant_vistas_aviso,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,edad,estado_estudio_Abandonado,estado_estudio_EnCurso,estado_estudio_Graduado,idaviso,idpostulante,nivel_laboral_Gerencia/AltaGerencia/Dirección,nivel_laboral_Jefe/Supervisor/Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior/Semi-Senior,nivel_laboral_nan,nombre_area,nombre_estudio_Doctorado,nombre_estudio_Master,nombre_estudio_Otro,nombre_estudio_Posgrado,nombre_estudio_Secundario,nombre_estudio_Terciario/Tecnico,nombre_estudio_Universitario,sepostulo,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,tipo_full,tipo_otro,tipo_part
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,0,1,0,1112390783,MVao3z9,0,0,0,0,0,1.0,NaN,0,0,0,0,0,0,1,True,1,0,0,False,True,False


In [18]:
X_test.head(1)

,idaviso,edad,nombre_estudio_Doctorado,nombre_estudio_Master,nombre_estudio_Otro,nombre_estudio_Posgrado,nombre_estudio_Secundario,nombre_estudio_Terciario/Tecnico,nombre_estudio_Universitario,estado_estudio_Abandonado,estado_estudio_EnCurso,estado_estudio_Graduado,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,cant_vistas_aviso,cant_postulaciones_aviso,tipo_full,tipo_part,tipo_otro,nivel_laboral_Gerencia/AltaGerencia/Dirección,nivel_laboral_Jefe/Supervisor/Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior/Semi-Senior,Ventas,Administración,Producción,Comercial,AtenciónalCliente,Recepcionista,CallCenter,Tesorería,Telemarketing,MantenimientoyLimpieza,Almacén/Depósito/Expedición,Contabilidad,RecursosHumanos,Gastronomia,Logística,Secretaria,AdministracióndePersonal,CréditosyCobranzas,Marketing,Legal
0,739260,42,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,2,0,True,False,False,0,1,0,0,0,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [5]:
X_test.drop(columns=['nombre_area', 'idpostulante'], inplace=True)

In [10]:
X_train.drop(columns=['nombre_area', 'idpostulante', 'sepostulo', 'nivel_laboral_nan'], inplace=True)

In [11]:
print(X_train.columns)
print(X_test.columns)

Index(['Administración', 'AdministracióndePersonal',
       'Almacén/Depósito/Expedición', 'AtenciónalCliente', 'CallCenter',
       'Comercial', 'Contabilidad', 'CréditosyCobranzas', 'Gastronomia',
       'Legal', 'Logística', 'MantenimientoyLimpieza', 'Marketing',
       'Producción', 'Recepcionista', 'RecursosHumanos', 'Secretaria',
       'Telemarketing', 'Tesorería', 'Ventas', 'cant_postulaciones_aviso',
       'cant_vistas_aviso', 'desc_1', 'desc_2', 'desc_3', 'desc_4', 'desc_5',
       'desc_6', 'desc_7', 'edad', 'estado_estudio_Abandonado',
       'estado_estudio_EnCurso', 'estado_estudio_Graduado', 'idaviso',
       'nivel_laboral_Gerencia/AltaGerencia/Dirección',
       'nivel_laboral_Jefe/Supervisor/Responsable', 'nivel_laboral_Junior',
       'nivel_laboral_Otro', 'nivel_laboral_Senior/Semi-Senior',
       'nombre_estudio_Doctorado', 'nombre_estudio_Master',
       'nombre_estudio_Otro', 'nombre_estudio_Posgrado',
       'nombre_estudio_Secundario', 'nombre_estudio_Terciari

In [ ]:
# Ordenamos las columnas para que queden igual
#X_train = X_train.reindex_axis(sorted(X_train.columns), axis=1)
#X_test = X_test.reindex_axis(sorted(X_test.columns), axis=1)
#X_train = X_train.reindex(sorted(X_train.columns), axis=1)
#X_test = X_test.reindex(sorted(X_test.columns), axis=1)

In [ ]:
#print(X_train.isnull().values.any())
#print(X_test.isnull().values.any())

In [12]:
print(X_train.shape)
print(X_test.shape)
print(X_train.columns)
print(X_test.columns)

(13097808, 52)
(100120, 52)
Index(['Administración', 'AdministracióndePersonal',
       'Almacén/Depósito/Expedición', 'AtenciónalCliente', 'CallCenter',
       'Comercial', 'Contabilidad', 'CréditosyCobranzas', 'Gastronomia',
       'Legal', 'Logística', 'MantenimientoyLimpieza', 'Marketing',
       'Producción', 'Recepcionista', 'RecursosHumanos', 'Secretaria',
       'Telemarketing', 'Tesorería', 'Ventas', 'cant_postulaciones_aviso',
       'cant_vistas_aviso', 'desc_1', 'desc_2', 'desc_3', 'desc_4', 'desc_5',
       'desc_6', 'desc_7', 'edad', 'estado_estudio_Abandonado',
       'estado_estudio_EnCurso', 'estado_estudio_Graduado', 'idaviso',
       'nivel_laboral_Gerencia/AltaGerencia/Dirección',
       'nivel_laboral_Jefe/Supervisor/Responsable', 'nivel_laboral_Junior',
       'nivel_laboral_Otro', 'nivel_laboral_Senior/Semi-Senior',
       'nombre_estudio_Doctorado', 'nombre_estudio_Master',
       'nombre_estudio_Otro', 'nombre_estudio_Posgrado',
       'nombre_estudio_Secundari

In [13]:
X_train.dtypes

Administración                                      bool
AdministracióndePersonal                            bool
Almacén/Depósito/Expedición                         bool
AtenciónalCliente                                   bool
CallCenter                                          bool
Comercial                                           bool
Contabilidad                                        bool
CréditosyCobranzas                                  bool
Gastronomia                                         bool
Legal                                               bool
Logística                                           bool
MantenimientoyLimpieza                              bool
Marketing                                           bool
Producción                                          bool
Recepcionista                                       bool
RecursosHumanos                                     bool
Secretaria                                          bool
Telemarketing                  

In [14]:
X_test.dtypes

idaviso                                          int64
edad                                             int64
nombre_estudio_Doctorado                         int64
nombre_estudio_Master                            int64
nombre_estudio_Otro                              int64
nombre_estudio_Posgrado                          int64
nombre_estudio_Secundario                        int64
nombre_estudio_Terciario/Tecnico                 int64
nombre_estudio_Universitario                     int64
estado_estudio_Abandonado                        int64
estado_estudio_EnCurso                           int64
estado_estudio_Graduado                          int64
sexo_FEM                                         int64
sexo_MASC                                        int64
sexo_NO_DECLARA                                  int64
desc_1                                           int64
desc_2                                           int64
desc_3                                           int64
desc_4    

# Checkpoint 

In [16]:
X_train.head(10000000).to_csv('../data/X_train_10000000.csv', index=False, sep=',', encoding='utf-8')
target.head(10000000).to_csv('../data/target_10000000.csv', index=False, sep=',', encoding='utf-8')

In [2]:
X_train = pd.read_csv('../data/X_train_10000000.csv', sep=',', encoding='utf-8')
target = pd.read_csv('../data/target_10000000.csv', sep=',', encoding='utf-8')

In [3]:
X_test = pd.read_csv('../data/X_test.csv', sep=',', encoding='utf-8')

In [6]:
print(X_train.shape)
print(X_test.shape)
print(target.shape)

(10000000, 52)
(100000, 52)
(10000000, 1)


In [28]:
X_test.head(1)

,Administración,AdministracióndePersonal,Almacén/Depósito/Expedición,AtenciónalCliente,CallCenter,Comercial,Contabilidad,CréditosyCobranzas,Gastronomia,Legal,Logística,MantenimientoyLimpieza,Marketing,Producción,Recepcionista,RecursosHumanos,Secretaria,Telemarketing,Tesorería,Ventas,cant_postulaciones_aviso,cant_vistas_aviso,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,edad,estado_estudio_Abandonado,estado_estudio_EnCurso,estado_estudio_Graduado,idaviso,nivel_laboral_Gerencia/AltaGerencia/Dirección,nivel_laboral_Jefe/Supervisor/Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior/Semi-Senior,nombre_estudio_Doctorado,nombre_estudio_Master,nombre_estudio_Otro,nombre_estudio_Posgrado,nombre_estudio_Secundario,nombre_estudio_Terciario/Tecnico,nombre_estudio_Universitario,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,tipo_full,tipo_otro,tipo_part
0,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,2,False,False,False,False,True,False,False,42,False,False,True,739260,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False


In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 52 columns):
Administración                                   bool
AdministracióndePersonal                         bool
Almacén/Depósito/Expedición                      bool
AtenciónalCliente                                bool
CallCenter                                       bool
Comercial                                        bool
Contabilidad                                     bool
CréditosyCobranzas                               bool
Gastronomia                                      bool
Legal                                            bool
Logística                                        bool
MantenimientoyLimpieza                           bool
Marketing                                        bool
Producción                                       bool
Recepcionista                                    bool
RecursosHumanos                                  bool
Secretaria            

In [5]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 52 columns):
Administración                                   100000 non-null bool
AdministracióndePersonal                         100000 non-null bool
Almacén/Depósito/Expedición                      100000 non-null bool
AtenciónalCliente                                100000 non-null bool
CallCenter                                       100000 non-null bool
Comercial                                        100000 non-null bool
Contabilidad                                     100000 non-null bool
CréditosyCobranzas                               100000 non-null bool
Gastronomia                                      100000 non-null bool
Legal                                            100000 non-null bool
Logística                                        100000 non-null bool
MantenimientoyLimpieza                           100000 non-null bool
Marketing                                        1

In [6]:
X_train.dtypes

Administración                                    bool
AdministracióndePersonal                          bool
Almacén/Depósito/Expedición                       bool
AtenciónalCliente                                 bool
CallCenter                                        bool
Comercial                                         bool
Contabilidad                                      bool
CréditosyCobranzas                                bool
Gastronomia                                       bool
Legal                                             bool
Logística                                         bool
MantenimientoyLimpieza                            bool
Marketing                                         bool
Producción                                        bool
Recepcionista                                     bool
RecursosHumanos                                   bool
Secretaria                                        bool
Telemarketing                                     bool
Tesorería 

In [7]:
X_train.desc_1.fillna(0, inplace=True)
X_train.desc_2.fillna(0, inplace=True)
X_train.desc_3.fillna(0, inplace=True)
X_train.desc_4.fillna(0, inplace=True)
X_train.desc_5.fillna(0, inplace=True)
X_train.desc_6.fillna(0, inplace=True)
X_train.desc_7.fillna(0, inplace=True)

In [8]:
X_train['desc_1'] = X_train['desc_1'].astype(bool)
X_train['desc_2'] = X_train['desc_2'].astype(bool)
X_train['desc_3'] = X_train['desc_3'].astype(bool)
X_train['desc_4'] = X_train['desc_4'].astype(bool)
X_train['desc_5'] = X_train['desc_5'].astype(bool)
X_train['desc_6'] = X_train['desc_6'].astype(bool)
X_train['desc_7'] = X_train['desc_7'].astype(bool)

X_train['estado_estudio_Abandonado'] = X_train['estado_estudio_Abandonado'].astype(bool)
X_train['estado_estudio_EnCurso'] = X_train['estado_estudio_EnCurso'].astype(bool)
X_train['estado_estudio_Graduado'] = X_train['estado_estudio_Graduado'].astype(bool)
X_train['nivel_laboral_Gerencia/AltaGerencia/Dirección'] = X_train['nivel_laboral_Gerencia/AltaGerencia/Dirección'].astype(bool)
X_train['nivel_laboral_Jefe/Supervisor/Responsable'] = X_train['nivel_laboral_Jefe/Supervisor/Responsable'].astype(bool)
X_train['nivel_laboral_Junior'] = X_train['nivel_laboral_Junior'].astype(bool)
X_train['nivel_laboral_Otro'] = X_train['nivel_laboral_Otro'].astype(bool)
X_train['nivel_laboral_Senior/Semi-Senior'] = X_train['nivel_laboral_Senior/Semi-Senior'].astype(bool)
X_train['nombre_estudio_Doctorado'] = X_train['nombre_estudio_Doctorado'].astype(bool)
X_train['nombre_estudio_Master'] = X_train['nombre_estudio_Master'].astype(bool)
X_train['nombre_estudio_Otro'] = X_train['nombre_estudio_Otro'].astype(bool)
X_train['nombre_estudio_Posgrado'] = X_train['nombre_estudio_Posgrado'].astype(bool)
X_train['nombre_estudio_Secundario'] = X_train['nombre_estudio_Secundario'].astype(bool)
X_train['nombre_estudio_Terciario/Tecnico'] = X_train['nombre_estudio_Terciario/Tecnico'].astype(bool)
X_train['nombre_estudio_Universitario'] = X_train['nombre_estudio_Universitario'].astype(bool)
X_train['sexo_FEM'] = X_train['sexo_FEM'].astype(bool)
X_train['sexo_MASC'] = X_train['sexo_MASC'].astype(bool)
X_train['sexo_NO_DECLARA'] = X_train['sexo_NO_DECLARA'].astype(bool)

In [9]:
X_train.dtypes

Administración                                    bool
AdministracióndePersonal                          bool
Almacén/Depósito/Expedición                       bool
AtenciónalCliente                                 bool
CallCenter                                        bool
Comercial                                         bool
Contabilidad                                      bool
CréditosyCobranzas                                bool
Gastronomia                                       bool
Legal                                             bool
Logística                                         bool
MantenimientoyLimpieza                            bool
Marketing                                         bool
Producción                                        bool
Recepcionista                                     bool
RecursosHumanos                                   bool
Secretaria                                        bool
Telemarketing                                     bool
Tesorería 

In [10]:
X_test.dtypes

idaviso                                          int64
edad                                             int64
nombre_estudio_Doctorado                          bool
nombre_estudio_Master                             bool
nombre_estudio_Otro                               bool
nombre_estudio_Posgrado                           bool
nombre_estudio_Secundario                         bool
nombre_estudio_Terciario/Tecnico                  bool
nombre_estudio_Universitario                      bool
estado_estudio_Abandonado                         bool
estado_estudio_EnCurso                            bool
estado_estudio_Graduado                           bool
sexo_FEM                                          bool
sexo_MASC                                         bool
sexo_NO_DECLARA                                   bool
desc_1                                            bool
desc_2                                            bool
desc_3                                            bool
desc_4    

In [11]:
X_test.desc_1.fillna(0, inplace=True)
X_test.desc_2.fillna(0, inplace=True)
X_test.desc_3.fillna(0, inplace=True)
X_test.desc_4.fillna(0, inplace=True)
X_test.desc_5.fillna(0, inplace=True)
X_test.desc_6.fillna(0, inplace=True)
X_test.desc_7.fillna(0, inplace=True)

In [12]:
X_test['desc_1'] = X_test['desc_1'].astype(bool)
X_test['desc_2'] = X_test['desc_2'].astype(bool)
X_test['desc_3'] = X_test['desc_3'].astype(bool)
X_test['desc_4'] = X_test['desc_4'].astype(bool)
X_test['desc_5'] = X_test['desc_5'].astype(bool)
X_test['desc_6'] = X_test['desc_6'].astype(bool)
X_test['desc_7'] = X_test['desc_7'].astype(bool)

X_test['estado_estudio_Abandonado'] = X_test['estado_estudio_Abandonado'].astype(bool)
X_test['estado_estudio_EnCurso'] = X_test['estado_estudio_EnCurso'].astype(bool)
X_test['estado_estudio_Graduado'] = X_test['estado_estudio_Graduado'].astype(bool)
X_test['nivel_laboral_Gerencia/AltaGerencia/Dirección'] = X_test['nivel_laboral_Gerencia/AltaGerencia/Dirección'].astype(bool)
X_test['nivel_laboral_Jefe/Supervisor/Responsable'] = X_test['nivel_laboral_Jefe/Supervisor/Responsable'].astype(bool)
X_test['nivel_laboral_Junior'] = X_test['nivel_laboral_Junior'].astype(bool)
X_test['nivel_laboral_Otro'] = X_test['nivel_laboral_Otro'].astype(bool)
X_test['nivel_laboral_Senior/Semi-Senior'] = X_test['nivel_laboral_Senior/Semi-Senior'].astype(bool)
X_test['nombre_estudio_Doctorado'] = X_test['nombre_estudio_Doctorado'].astype(bool)
X_test['nombre_estudio_Master'] = X_test['nombre_estudio_Master'].astype(bool)
X_test['nombre_estudio_Otro'] = X_test['nombre_estudio_Otro'].astype(bool)
X_test['nombre_estudio_Posgrado'] = X_test['nombre_estudio_Posgrado'].astype(bool)
X_test['nombre_estudio_Secundario'] = X_test['nombre_estudio_Secundario'].astype(bool)
X_test['nombre_estudio_Terciario/Tecnico'] = X_test['nombre_estudio_Terciario/Tecnico'].astype(bool)
X_test['nombre_estudio_Universitario'] = X_test['nombre_estudio_Universitario'].astype(bool)
X_test['sexo_FEM'] = X_test['sexo_FEM'].astype(bool)
X_test['sexo_MASC'] = X_test['sexo_MASC'].astype(bool)
X_test['sexo_NO_DECLARA'] = X_test['sexo_NO_DECLARA'].astype(bool)

In [13]:
X_test.dtypes

idaviso                                          int64
edad                                             int64
nombre_estudio_Doctorado                          bool
nombre_estudio_Master                             bool
nombre_estudio_Otro                               bool
nombre_estudio_Posgrado                           bool
nombre_estudio_Secundario                         bool
nombre_estudio_Terciario/Tecnico                  bool
nombre_estudio_Universitario                      bool
estado_estudio_Abandonado                         bool
estado_estudio_EnCurso                            bool
estado_estudio_Graduado                           bool
sexo_FEM                                          bool
sexo_MASC                                         bool
sexo_NO_DECLARA                                   bool
desc_1                                            bool
desc_2                                            bool
desc_3                                            bool
desc_4    

In [18]:
target.shape

(10000000, 1)

In [17]:
X_test

,idaviso,edad,nombre_estudio_Doctorado,nombre_estudio_Master,nombre_estudio_Otro,nombre_estudio_Posgrado,nombre_estudio_Secundario,nombre_estudio_Terciario/Tecnico,nombre_estudio_Universitario,estado_estudio_Abandonado,estado_estudio_EnCurso,estado_estudio_Graduado,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,cant_vistas_aviso,cant_postulaciones_aviso,tipo_full,tipo_part,tipo_otro,nivel_laboral_Gerencia/AltaGerencia/Dirección,nivel_laboral_Jefe/Supervisor/Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior/Semi-Senior,Ventas,Administración,Producción,Comercial,AtenciónalCliente,Recepcionista,CallCenter,Tesorería,Telemarketing,MantenimientoyLimpieza,Almacén/Depósito/Expedición,Contabilidad,RecursosHumanos,Gastronomia,Logística,Secretaria,AdministracióndePersonal,CréditosyCobranzas,Marketing,Legal
0,739260,42,True,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,2,0,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,739260,31,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,True,False,False,False,2,0,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,739260,36,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,True,False,False,False,2,0,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,758580,69,False,False,False,True,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False,False,1,0,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,758580,32,False,False,False,False,False,False,True,False,True,False,True,False,False,False,True,False,False,False,False,False,1,0,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,758580,28,False,False,False,False,False,False,True,True,False,False,False,True,False,False,True,False,False,False,False,False,1,0,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,776420,29,False,False,False,False,False,False,True,False,False,True,False,True,False,False,False,False,False,False,True,False,1,0,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,776420,54,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,True,False,1,0,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,776420,28,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,False,True,False,1,0,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,820850,38,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,1,0,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False


In [32]:
X_train.head(10000000).to_csv('../data/X_train_10000000.csv', index=False, sep=',', encoding='utf-8')
X_test.to_csv('../data/X_test.csv', index=False, sep=',', encoding='utf-8')

In [33]:
print(len(X_train))
print(len(target))

10000000
10000000


# Exportamos una muestra mas "chica" pero significativa de los datos para hacer feature selection y busqueda de hiperparametros

In [33]:
X_train.head(5000000).to_csv('../data/X_train_5000000.csv', index=False, sep=',', encoding='utf-8')
target.head(5000000).to_csv('../data/target_5000000.csv', index=False, sep=',', encoding='utf-8')

In [2]:
X_train = pd.read_csv('../data/X_train_5000000.csv', sep=',', encoding='utf-8')
target = pd.read_csv('../data/target_5000000.csv', sep=',', encoding='utf-8')

In [3]:
print(X_train.shape)
print(target.shape)

(5000000, 52)
(5000000, 1)


## **Rankeo de algoritmos**

In [4]:
print(len(target[target['sepostulo'] == False]))
print(len(target[target['sepostulo'] == True]))

2479725
2520275


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

models = []
models.append(('DecTree',DecisionTreeClassifier()))
models.append(('LogReg',LogisticRegression()))
models.append(('KNN',KNeighborsClassifier()))
models.append(('LDA',LinearDiscriminantAnalysis()))
models.append(('GaussianNB',GaussianNB()))
#models.append(('SVC',SVC())) # no termina
models.append(('MLP',MLPClassifier(max_iter=200))) # Ver si no es muy largo


In [40]:
%%time
scores = []
for name,model in models:
    print('################################',name,'################################')
    #score = cv_score(model, X_train, target.values.ravel(), cv=kfold)
    score = cv_score(model, X_train, target.values, cv=kfold)
    scores.append((name,score))
    
print('\n============== RANKING =================')
for name,score in sorted(scores,key=lambda x: x[1],reverse=True):
    print(name,':',score)


============== RANKING =================
KNN : 0.8709368895023483
LogReg : 0.7915612184155507
DecTree : 0.7861835802608504


**El que mejor score da es KNN**

# Feature Selection

In [8]:
scoring = 'roc_auc'
seed = 7
def select_features(model,X,y):
    from sklearn.feature_selection import RFECV
    rfecv = RFECV(estimator=model, n_jobs=1, scoring=scoring, verbose=3)
    rfecv.fit(X, y)
    return rfecv.support_

In [ ]:
%%time
best_features_mask = []
for name,model in models:
    print('##################################',name,'##################################')
    features = select_features(model, X_train, target.values.ravel())
    best_features_mask.append((name, features))

################################## DecTree ##################################
Fitting estimator with 52 features.
Fitting estimator with 51 features.
Fitting estimator with 50 features.
Fitting estimator with 49 features.
Fitting estimator with 48 features.
Fitting estimator with 47 features.
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with

In [ ]:
for model_name,features_mask in best_features_mask:
    print('=================================',model_name,'=================================')
    print('number of features:',len(X_train.columns[features_mask]))
    print(X_train.columns[features_mask])

# Grid Search

In [ ]:
scoring = 'roc_auc'
seed = 7
def select_params(model,params,X,y):
    from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
    gs = GridSearchCV(model,params,cv=kfold,n_jobs=-1,verbose=3,scoring=scoring)
    #rs = RandomizedSearchCV(model,params, cv=kfold, n_jobs=2, verbose=3, scoring=scoring, random_state=seed)
    gs.fit(X,y)
    #rs.fit(X,y)
    return gs.best_params_
    #return rs.best_params_

In [ ]:
%%time
from scipy.stats import randint, expon ,uniform, gamma, alpha
params = {}
params.update({'DecTree':[{'min_samples_split': [2, 3, 4]}]})
params.update({'LogReg':[]})
params.update({'KNN':[{'leaf_size': [20,30,40]}, {'p': [1,2,3]}, {'weights': ['uniform', 'distance']}]})
params.update({'LDA':[]})
params.update({'GaussianNB':[]})
params.update({'SVC':[]})
params.update({'MLP':[]})

best_params = []
for name, model in models:
    params = select_params(model,params[name], X_train, target.values.ravel())
    best_params.append((name, params))

# Cross-validation

In [36]:
from sklearn.model_selection import cross_val_score, KFold
# seed para generar los mismos fold en cada cv
seed = 7
kfold = KFold(3,random_state=seed) # 3 sino tarda demasiado
scoring = 'roc_auc'
def cv_score(model,X,y,cv):
    #return cross_val_score(model,X,y,n_jobs=-1,cv=cv,scoring=scoring,verbose=3).mean()
    return cross_val_score(model,X,y,n_jobs=1,cv=cv,scoring=scoring,verbose=3).mean()

**Standardization & normalization**

In [25]:
from sklearn.preprocessing import scale, normalize
X_train_std = scale(X_train)
X_test_std = scale(X_test)
X_train_nrm = normalize(X_train)
X_test_nrm = normalize(X_test)

/home/tbotalla/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


** Train test split**

In [33]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, target, test_size = 0.33, random_state = 0)
print("X_train : " + str(X_train.shape))
print("X_test : " + str(X_test.shape))
print("y_train : " + str(y_train.shape))
print("y_test : " + str(y_test.shape))

X_train : (8775627, 46)
X_test : (4322325, 46)
y_train : (8775627, 1)
y_test : (4322325, 1)


# Predicciones

In [14]:
def predict(model,X_train,X_test,y):
    model.fit(X_train,y.values.ravel())
    return model.predict(X_test)

def pred_to_csv(prediction,name):
    test = pd.read_csv('test_.csv')
    df = pd.DataFrame(prediction,columns={'sepostulo'})
    df['id'] = test.id
    df = df.loc[:,['id','sepostulo']]
    df.sepostulo = df.sepostulo.astype(int)
    df.to_csv('pred_' + name + '.csv',index=False)

In [17]:
%%time
from sklearn.linear_model import SGDClassifier

pred = predict(SGDClassifier(), X_train, X_test, target)
pred_to_csv(pred,'SGD')

/home/tbotalla/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


CPU times: user 1min 4s, sys: 26.1 s, total: 1min 30s
Wall time: 6min 57s


In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier

pred = predict(DecisionTreeClassifier(), X_train, X_test, target)
pred_to_csv(pred,'DecisionTreeClassifier') # 0.7 score en kaggle

CPU times: user 5min 48s, sys: 20.5 s, total: 6min 9s
Wall time: 10min


In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier

pred = predict(KNeighborsClassifier(), X_train, X_test, target)
pred_to_csv(pred,'KNeighborsClassifier') # 0.76 score en kaggle

In [12]:
print(X_train.shape)
print(X_test.shape)
print(target.shape)

(12000000, 45)
(100000, 45)
(12000000, 1)


In [16]:
target

,sepostulo
0,True
1,False
2,True
3,True
4,True
5,False
6,True
7,True
8,True
9,True
